In [7]:
%%writefile main.cu
#include <cuda_runtime.h> // cuda api
#include <device_launch_parameters.h> // threadIdx, blockIdx
#include <iostream> // cout
#include <vector> // vector
#include <algorithm> // sort, heap, is_sorted
#include <random> // рандом
#include <chrono> // время

using namespace std; // чтоб не писать std::

const int INF = 2147483647; // значение-заглушка
const int CHUNK = 256; // сколько элементов сортим в одном блоке

void cuda_ok(cudaError_t err) { // проверка ошибок cuda
    if (err != cudaSuccess) { // если вернулась ошибка
        cout << "ошибка cuda: " << cudaGetErrorString(err) << "\n"; // печатаем причину
        exit(1); // дальше смысла нет
    }
}

void cpu_merge(vector<int>& a) { // merge sort на cpu
    int n = (int)a.size(); // размер
    vector<int> tmp(n); // временный массив
    for (int w = 1; w < n; w *= 2) { // ширина кусков
        for (int i = 0; i < n; i += 2 * w) { // берем пару кусков
            int l = i; // старт слева
            int m = min(i + w, n); // середина
            int r = min(i + 2 * w, n); // конец
            int p = l; // идем по левому куску
            int q = m; // идем по правому куску
            int k = l; // куда пишем
            while (p < m && q < r) { // пока оба куска не кончились
                if (a[p] <= a[q]) tmp[k++] = a[p++]; // берем меньшее слева
                else tmp[k++] = a[q++]; // или справа
            }
            while (p < m) tmp[k++] = a[p++]; // дописываем остаток слева
            while (q < r) tmp[k++] = a[q++]; // дописываем остаток справа
        }
        a.swap(tmp); // теперь tmp стал новым a
    }
}

void cpu_quick(vector<int>& a) { // quick на cpu
    sort(a.begin(), a.end()); // std::sort
}

void cpu_heap(vector<int>& a) { // heap на cpu
    make_heap(a.begin(), a.end()); // собираем кучу
    sort_heap(a.begin(), a.end()); // вытаскиваем элементы
}

__global__ void merge_pass(const int* in, int* out, int n, int w) { // один проход слияния
    int pair_id = blockIdx.x; // номер пары кусков
    int start = pair_id * (2 * w); // старт пары
    if (start >= n) return; // если вышли за массив

    int mid = start + w; // середина
    int end = start + 2 * w; // конец
    if (mid > n) mid = n; // не выходим за границы
    if (end > n) end = n; // не выходим за границы

    if (threadIdx.x == 0) { // делаем слияние одним потоком (проще)
        int i = start; // левый индекс
        int j = mid; // правый индекс
        int k = start; // куда пишем
        while (i < mid && j < end) { // пока есть элементы и слева и справа
            if (in[i] <= in[j]) out[k++] = in[i++]; // берем меньшее
            else out[k++] = in[j++]; // или справа
        }
        while (i < mid) out[k++] = in[i++]; // хвост слева
        while (j < end) out[k++] = in[j++]; // хвост справа
    }
}

__global__ void chunk_bitonic(const int* in, int* out, int n) { // сортим один chunk битоником
    __shared__ int s[CHUNK]; // общий буфер блока
    int tid = threadIdx.x; // поток в блоке
    int base = blockIdx.x * CHUNK; // начало куска
    int idx = base + tid; // глобальный индекс

    s[tid] = (idx < n) ? in[idx] : INF; // грузим данные или заглушку
    __syncthreads(); // ждем пока все загрузили

    for (int k = 2; k <= CHUNK; k <<= 1) { // размер битонной группы
        for (int j = k >> 1; j > 0; j >>= 1) { // шаг сравнения
            int ixj = tid ^ j; // парный индекс
            if (ixj > tid) { // чтобы не делать пару два раза
                bool up = ((tid & k) == 0); // направление сортировки
                int a = s[tid]; // наше
                int b = s[ixj]; // парное
                if (up) { if (a > b) { s[tid] = b; s[ixj] = a; } } // по возрастанию
                else { if (a < b) { s[tid] = b; s[ixj] = a; } } // по убыванию
            }
            __syncthreads(); // синхронизация после шага
        }
    }

    if (idx < n) out[idx] = s[tid]; // сохраняем назад
}

__device__ void ins_sort(int* a, int n) { // insertion sort (для маленького куска)
    for (int i = 1; i < n; i++) { // идем с 1
        int key = a[i]; // текущий
        int j = i - 1; // слева
        while (j >= 0 && a[j] > key) { // сдвигаем пока больше key
            a[j + 1] = a[j]; // сдвиг вправо
            j--; // влево
        }
        a[j + 1] = key; // вставили
    }
}

__global__ void chunk_quick_simple(const int* in, int* out, int n) { // "quick" на gpu (по факту вставки)
    __shared__ int s[CHUNK]; // общий буфер блока
    int tid = threadIdx.x; // поток
    int base = blockIdx.x * CHUNK; // начало
    int idx = base + tid; // индекс

    s[tid] = (idx < n) ? in[idx] : INF; // грузим
    __syncthreads(); // ждем

    if (tid == 0) ins_sort(s, CHUNK); // сортит один поток, чтобы было просто
    __syncthreads(); // ждем

    if (idx < n) out[idx] = s[tid]; // сохраняем
}

__device__ void sift_down(int* a, int start, int end) { // просеивание вниз для кучи
    int root = start; // корень
    while (true) { // пока можно
        int child = 2 * root + 1; // левый ребенок
        if (child >= end) return; // детей нет
        int best = root; // кто максимум
        if (a[best] < a[child]) best = child; // сравнили с левым
        if (child + 1 < end && a[best] < a[child + 1]) best = child + 1; // сравнили с правым
        if (best == root) return; // уже нормально
        int temp = a[root]; a[root] = a[best]; a[best] = temp; // обмен
        root = best; // идем ниже
    }
}

__device__ void heap_sort_chunk(int* a, int n) { // heapsort для одного куска
    for (int i = n / 2 - 1; i >= 0; i--) sift_down(a, i, n); // строим max-heap
    for (int end = n - 1; end > 0; end--) { // вытаскиваем максимум
        int temp = a[0]; a[0] = a[end]; a[end] = temp; // обмен с концом
        sift_down(a, 0, end); // чиним кучу
    }
}

__global__ void chunk_heap_simple(const int* in, int* out, int n) { // heap сортировка чанка на gpu
    __shared__ int s[CHUNK]; // shared
    int tid = threadIdx.x; // поток
    int base = blockIdx.x * CHUNK; // начало
    int idx = base + tid; // индекс

    s[tid] = (idx < n) ? in[idx] : INF; // грузим
    __syncthreads(); // ждем

    if (tid == 0) heap_sort_chunk(s, CHUNK); // один поток сортит chunk
    __syncthreads(); // ждем

    if (idx < n) out[idx] = s[tid]; // сохраняем
}

float gpu_sort(int mode, const vector<int>& input) { // mode: 0 merge, 1 quick, 2 heap
    int n = (int)input.size(); // размер
    int* d_in = nullptr; // данные на gpu
    int* d_tmp = nullptr; // временный буфер на gpu

    cuda_ok(cudaMalloc(&d_in, n * (int)sizeof(int))); // выделили вход
    cuda_ok(cudaMalloc(&d_tmp, n * (int)sizeof(int))); // выделили tmp
    cuda_ok(cudaMemcpy(d_in, input.data(), n * (int)sizeof(int), cudaMemcpyHostToDevice)); // залили на gpu

    cudaEvent_t st, en; // события для таймера
    cuda_ok(cudaEventCreate(&st)); // старт
    cuda_ok(cudaEventCreate(&en)); // конец
    cuda_ok(cudaEventRecord(st)); // пошли

    int blocks = (n + CHUNK - 1) / CHUNK; // сколько блоков нужно

    if (mode == 0) chunk_bitonic<<<blocks, CHUNK>>>(d_in, d_tmp, n); // sort chunk для merge
    if (mode == 1) chunk_quick_simple<<<blocks, CHUNK>>>(d_in, d_tmp, n); // sort chunk для "quick"
    if (mode == 2) chunk_heap_simple<<<blocks, CHUNK>>>(d_in, d_tmp, n); // sort chunk для heap

    cuda_ok(cudaGetLastError()); // проверили запуск
    cuda_ok(cudaDeviceSynchronize()); // дождались конца

    int* src = d_tmp; // откуда читаем
    int* dst = d_in; // куда пишем

    for (int w = CHUNK; w < n; w *= 2) { // увеличиваем размер кусков
        int pairs = (n + (2 * w) - 1) / (2 * w); // сколько слияний
        merge_pass<<<pairs, 1>>>(src, dst, n, w); // сливаем
        cuda_ok(cudaGetLastError()); // проверили
        cuda_ok(cudaDeviceSynchronize()); // дождались
        int* t = src; src = dst; dst = t; // поменяли местами
    }

    if (src != d_in) cuda_ok(cudaMemcpy(d_in, src, n * (int)sizeof(int), cudaMemcpyDeviceToDevice)); // финал в d_in

    cuda_ok(cudaEventRecord(en)); // стоп
    cuda_ok(cudaEventSynchronize(en)); // дождались

    float ms = 0.0f; // сюда время
    cuda_ok(cudaEventElapsedTime(&ms, st, en)); // получили мс

    vector<int> out(n); // результат на cpu
    cuda_ok(cudaMemcpy(out.data(), d_in, n * (int)sizeof(int), cudaMemcpyDeviceToHost)); // забрали с gpu

    if (!is_sorted(out.begin(), out.end())) { // проверка на всякий
        cout << "gpu: массив не отсортировался\n"; // печать
        exit(1); // стоп
    }

    cuda_ok(cudaFree(d_in)); // чистим
    cuda_ok(cudaFree(d_tmp)); // чистим
    cuda_ok(cudaEventDestroy(st)); // чистим
    cuda_ok(cudaEventDestroy(en)); // чистим

    return ms; // время gpu
}

double cpu_time(void(*fn)(vector<int>&), const vector<int>& input) { // таймер для cpu
    vector<int> a = input; // копия
    auto s = chrono::high_resolution_clock::now(); // старт
    fn(a); // сортим
    auto e = chrono::high_resolution_clock::now(); // конец
    if (!is_sorted(a.begin(), a.end())) { // проверка
        cout << "cpu: массив не отсортировался\n"; // печать
        exit(1); // стоп
    }
    return chrono::duration<double, milli>(e - s).count(); // мс
}

void run_n(int n) { // тест на одном размере
    mt19937 gen(123); // фикс seed
    uniform_int_distribution<int> dist(0, 1000000); // диапазон чисел
    vector<int> a(n); // массив
    for (int i = 0; i < n; i++) a[i] = dist(gen); // заполнили

    cout << "\nразмер: " << n << "\n"; // вывод размера

    cout << "cpu merge: " << cpu_time(cpu_merge, a) << " мс\n"; // cpu merge
    cout << "cpu quick: " << cpu_time(cpu_quick, a) << " мс\n"; // cpu quick
    cout << "cpu heap : " << cpu_time(cpu_heap,  a) << " мс\n"; // cpu heap

    cuda_ok(cudaFree(0)); // небольшой прогрев gpu

    cout << "gpu merge: " << gpu_sort(0, a) << " мс\n"; // gpu merge
    cout << "gpu quick: " << gpu_sort(1, a) << " мс\n"; // gpu quick
    cout << "gpu heap : " << gpu_sort(2, a) << " мс\n"; // gpu heap
}

void practice3() { // практика 3
    run_n(10000); // 10к
    run_n(100000); // 100к
    run_n(1000000); // 1м
}

int main() { // main
    practice3(); // запуск
    return 0; // выход
}


Overwriting main.cu


In [8]:
!nvcc main.cu -O2 -arch=sm_75 -o main
!./main



размер: 10000
cpu merge: 1.92334 мс
cpu quick: 0.655967 мс
cpu heap : 2.43128 мс
gpu merge: 5.28995 мс
gpu quick: 7.2151 мс
gpu heap : 5.54435 мс

размер: 100000
cpu merge: 12.9039 мс
cpu quick: 7.94322 мс
cpu heap : 13.702 мс
gpu merge: 44.5539 мс
gpu quick: 50.4605 мс
gpu heap : 45.8896 мс

размер: 1000000
cpu merge: 154.929 мс
cpu quick: 91.1061 мс
cpu heap : 180.065 мс
gpu merge: 320.827 мс
gpu quick: 190.323 мс
gpu heap : 176.182 мс


In [3]:
!nvidia-smi

Tue Dec 30 09:55:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----